 - L1 & L2 的差別
    - [LINK](https://discuss.pytorch.org/t/how-is-your-experience-of-using-l1-regularization/153565)
 - 注意tensor和scalar<font color=yellow>不要混用</font>
 - 備註
    - 我沒有切Validation

In [ ]:
from torch import nn
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader
from torch.optim import Adam
from torchvision.transforms import ToTensor, Compose
from torch.nn import CrossEntropyLoss
import torch
from torch.nn.functional import one_hot
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# hyper-parameter
batch_size = 256
epochs = 100
lambd = 1
reg = 'L2' # None, 'L1', 'L2'

In [ ]:
train_data = FashionMNIST('data', train=True, transform=ToTensor()
                          , download=True)
test_data = FashionMNIST('data', train=False, transform=ToTensor()
                          , download=True)

train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
class Module(nn.Module):
    def __init__(self):
        super(Module, self).__init__()
        self.net = nn.Sequential(
            nn.Flatten(), 
            nn.LazyLinear(10)
        )
    def forward(self, X):
        return self.net(X)

In [ ]:
model = Module()
optimizer = Adam(model.parameters(), lr=0.003)
loss = CrossEntropyLoss()

In [ ]:
history = {'acc':[], 'val_acc':[], 'loss':[], 'val_loss':[]}
for i in range(epochs):
    print(f'==== Epoch {i + 1} start! ============================')
    model.train()
    training_acc = 0
    training_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        img, label = data
        output = model(img)
        output_loss = loss(output, label.long())
        # record
        training_loss = training_loss + output_loss.item() # 因為原本是tensor
        training_acc = training_acc + (output.argmax(dim=1) == label).sum().item() # 原本是tensor
        # Regularization
        # reg_loss = torch.tensor(0, dtype=torch.float32)
        # if reg == 'L1': # L1 Norm
        #     reg_loss = lambd * sum(param.norm(1) for param in model.parameters())
        # elif reg == 'L2': # L2 Norm
        #     reg_loss = lambd * sum(param.norm(2) for param in model.parameters())
        # output_loss = output_loss + reg_loss.detach()
        
        if reg == 'L1':
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            output_loss += lambd * l1_norm
        elif reg == 'L2':
            l2_norm = sum(p.pow(2).sum() for p in model.parameters())
            output_loss += lambd * l2_norm

        # BP
        output_loss.backward()
        optimizer.step()
    history['loss'].append(training_loss / len(train_data))
    history['acc'].append(training_acc / len(train_data))
    model.eval()
    test_acc = 0
    test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            img, label = data
            output = model(img)
            test_acc = test_acc + (output.argmax(dim=1) == label).sum().item()
            test_loss = test_loss + loss(output, label).item()
    history['val_acc'].append(test_acc / len(test_data))
    history['val_loss'].append(test_loss / len(test_data))
    print(f'Training Accuracy {training_acc / len(train_data):.4f}, training loss: {training_loss / len(train_data):.4f}')
    print(f'Test Accuracy: {test_acc / len(test_data):.4f}, test loss: {test_loss / len(test_data):.4f}')
    print('===================== End ========================')
    print()

In [ ]:
plt.clf(); plt.cla() # clean
plt.plot(history['acc'], 'b', label='training acc')
plt.plot(history['val_acc'], 'r', label='val acc')
plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.clf(); plt.cla() # clean
plt.plot(history['loss'], 'b', label='training loss')
plt.plot(history['val_loss'], 'r', label='val loss')
plt.title('Loss')
plt.legend()
plt.show()

 - 視覺化

In [ ]:
true_label = []
pred_label = []
with torch.no_grad():
    for data in test_loader:
        img, label = data
        output = model(img).argmax(1)
        true_label.extend(label.numpy())
        pred_label.extend(output.numpy())
        

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print('\033[33m' + 'Accuracy' + '\033[0m', accuracy_score(true_label, pred_label))
print('\033[33m' + 'Classification Report' + '\033[0m',
      classification_report(true_label, pred_label, target_names=test_data.classes), 
      sep='\n')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(true_label, pred_label)
plt.cla(); plt.clf()
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=test_data.classes, yticklabels=test_data.classes)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Fashion-MNIST Confusion Matrix")
plt.show()

In [ ]:
from torchsummary import summary
summary(model, (784, ))